In [4]:
# ============================================================
# CPU-SAFE TCN FOR ASL LANDMARKS (TRAINING) - NEW DATASET
# MODEL COMPLETELY UNCHANGED
# ============================================================

import numpy as np
from pathlib import Path
from collections import Counter
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR

# ============================================================
# CONFIG
# ============================================================

DATA_DIR = Path(r"E:\ASL_Citizen\NEW\preprocessed_approach1_shahd")
DEVICE = "cpu"

TARGET_FRAMES = 512
FEATURE_DIM   = 270

BATCH_SIZE = 8
EPOCHS = 90
LR = 3e-4
WEIGHT_DECAY = 1e-4

PATIENCE = 12
GRAD_CLIP = 1.0
LABEL_SMOOTH = 0.1

MODEL_SAVE_PATH = DATA_DIR / "tcn_best_cpu_approach1.pth"
LABEL_ENCODER_PATH = DATA_DIR / "label_encoder_approach1.npy"

# ============================================================
# LOAD FILES (NO MASKS IN THIS DATASET)
# ============================================================

files, labels = [], []

for f in DATA_DIR.glob("*.npy"):
    arr = np.load(f)

    if arr.shape != (TARGET_FRAMES, FEATURE_DIM):
        continue

    # Extract label correctly:
    # "ABOUT.npy" → ABOUT
    # "ABOUT 2.npy" → ABOUT
    label = f.stem.split(" ")[0]

    files.append(str(f))
    labels.append(label)

# Remove rare classes (same logic as original)
cnt = Counter(labels)
keep = [i for i, y in enumerate(labels) if cnt[y] >= 2]

files = [files[i] for i in keep]
labels = [labels[i] for i in keep]
print("Total samples after filtering:", len(files))
print("Number of classes:", len(set(labels)))
# Encode labels
le = LabelEncoder()
y = le.fit_transform(labels)
np.save(LABEL_ENCODER_PATH, le.classes_)
num_classes = len(le.classes_)

# ============================================================
# SPLITS
# ============================================================

X_tr, X_tmp, y_tr, y_tmp = train_test_split(
    files, y, test_size=0.2, stratify=y, random_state=42
)

X_val, X_te, y_val, y_te = train_test_split(
    X_tmp, y_tmp, test_size=0.5, stratify=y_tmp, random_state=42
)

# ============================================================
# DATASET (GENERATES DUMMY MASK = ALL ONES)
# ============================================================

class ASLDataset(Dataset):
    def __init__(self, files, labels):
        self.files = files
        self.labels = labels

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        x = np.load(self.files[idx])
        x = torch.from_numpy(x).float().transpose(0, 1)  # (C, T)

        # Create FULL mask (since no padding in this dataset)
        m = torch.ones(TARGET_FRAMES).float()

        y = torch.tensor(self.labels[idx])
        return x, m, y

train_loader = DataLoader(ASLDataset(X_tr, y_tr), BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(ASLDataset(X_val, y_val), BATCH_SIZE)
test_loader  = DataLoader(ASLDataset(X_te, y_te), BATCH_SIZE)

# ============================================================
# MODEL (UNCHANGED EXACTLY)
# ============================================================

class TemporalBlock(nn.Module):
    def __init__(self, ic, oc, d):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv1d(ic, oc, 3, padding=d, dilation=d),
            nn.BatchNorm1d(oc),
            nn.ReLU(),
            nn.Conv1d(oc, oc, 3, padding=d, dilation=d),
            nn.BatchNorm1d(oc),
            nn.ReLU(),
        )
        self.res = nn.Conv1d(ic, oc, 1) if ic != oc else nn.Identity()

    def forward(self, x):
        y = self.net(x)
        y = y[..., :x.size(2)]
        return y + self.res(x)

class TCN(nn.Module):
    def __init__(self):
        super().__init__()
        chans = [192, 192, 192, 192]
        layers = []
        for i, c in enumerate(chans):
            layers.append(TemporalBlock(
                FEATURE_DIM if i == 0 else chans[i-1],
                c, 2 ** i
            ))
        self.tcn = nn.Sequential(*layers)
        self.fc = nn.Linear(chans[-1], num_classes)

    def masked_pool(self, x, m):
        m = m.unsqueeze(1)
        return (x * m).sum(2) / (m.sum(2) + 1e-6)

    def forward(self, x, m):
        x = self.tcn(x)
        x = self.masked_pool(x, m)
        return self.fc(x)

model = TCN().to(DEVICE)

# ============================================================
# LOSS (UNCHANGED)
# ============================================================

weights = compute_class_weight("balanced", classes=np.unique(y_tr), y=y_tr)
weights = torch.tensor(weights).float()

class SmoothCE(nn.Module):
    def __init__(self, eps=0.1):
        super().__init__()
        self.eps = eps

    def forward(self, logits, target):
        n = logits.size(1)
        logp = torch.log_softmax(logits, 1)
        y = torch.zeros_like(logp).fill_(self.eps / n)
        y.scatter_(1, target.unsqueeze(1), 1 - self.eps)
        return -(y * logp).sum(1).mean()

criterion = SmoothCE(LABEL_SMOOTH)
optimizer = torch.optim.AdamW(model.parameters(), LR, weight_decay=WEIGHT_DECAY)
scheduler = CosineAnnealingLR(optimizer, EPOCHS)

# ============================================================
# TRAIN
# ============================================================

def run(loader, train=True):
    model.train() if train else model.eval()
    loss_sum, correct, total = 0, 0, 0

    with torch.set_grad_enabled(train):
        for x, m, y in loader:
            x, m, y = x.to(DEVICE), m.to(DEVICE), y.to(DEVICE)

            out = model(x, m)
            loss = criterion(out, y)

            if train:
                optimizer.zero_grad()
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), GRAD_CLIP)
                optimizer.step()

            loss_sum += loss.item() * y.size(0)
            correct += (out.argmax(1) == y).sum().item()
            total += y.size(0)

    return loss_sum / total, correct / total

best, patience = -1e9, 0

for e in range(EPOCHS):
    tr_l, tr_a = run(train_loader, True)
    va_l, va_a = run(val_loader, False)
    scheduler.step()

    metric = va_a - va_l

    print(f"E{e+1:03d} | TL {tr_l:.3f} TA {tr_a:.3f} | VL {va_l:.3f} VA {va_a:.3f}")

    if metric > best:
        best = metric
        patience = 0
        torch.save(model.state_dict(), MODEL_SAVE_PATH)
    else:
        patience += 1
        if patience >= PATIENCE:
            break

# ============================================================
# TEST EVALUATION
# ============================================================

model.load_state_dict(torch.load(MODEL_SAVE_PATH, map_location=DEVICE))
model.eval()

test_loss, test_acc = run(test_loader, train=False)

print("\n==============================")
print(f"TEST LOSS: {test_loss:.4f}")
print(f"TEST ACC : {test_acc:.4f}")
print("==============================")

Total samples after filtering: 5568
Number of classes: 146
E001 | TL 4.487 TA 0.084 | VL 4.016 VA 0.165
E002 | TL 3.675 TA 0.187 | VL 3.396 VA 0.233
E003 | TL 3.291 TA 0.267 | VL 3.242 VA 0.293
E004 | TL 3.029 TA 0.341 | VL 3.461 VA 0.228
E005 | TL 2.854 TA 0.382 | VL 3.251 VA 0.289
E006 | TL 2.683 TA 0.439 | VL 2.672 VA 0.440
E007 | TL 2.555 TA 0.485 | VL 3.167 VA 0.352
E008 | TL 2.418 TA 0.531 | VL 2.814 VA 0.463
E009 | TL 2.317 TA 0.563 | VL 3.425 VA 0.300
E010 | TL 2.212 TA 0.590 | VL 3.434 VA 0.329
E011 | TL 2.140 TA 0.629 | VL 2.431 VA 0.566
E012 | TL 2.094 TA 0.636 | VL 2.471 VA 0.576
E013 | TL 2.024 TA 0.670 | VL 4.161 VA 0.262
E014 | TL 1.959 TA 0.685 | VL 2.738 VA 0.499
E015 | TL 1.903 TA 0.699 | VL 2.432 VA 0.582
E016 | TL 1.857 TA 0.726 | VL 2.145 VA 0.654
E017 | TL 1.807 TA 0.740 | VL 2.535 VA 0.539
E018 | TL 1.771 TA 0.750 | VL 2.152 VA 0.662
E019 | TL 1.729 TA 0.771 | VL 2.453 VA 0.573
E020 | TL 1.696 TA 0.788 | VL 2.746 VA 0.508
E021 | TL 1.653 TA 0.793 | VL 2.104 VA 0.